<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/exports/exporting_to_CSV_tutorial.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/exports/exporting_to_CSV_tutorial.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Export to CSV or Pandas Format

This notebook serves as a simplified How-To guide and provides examples of converting Labelbox export JSON to a CSV and [Pandas](https://github.com/Labelbox/labelpandas) friendly format.  

## Advance Approach

For a more abstracted approach please visit our [LabelPandas](https://github.com/Labelbox/labelpandas) library. You can use this library to abstract the steps that are about to be shown. In addition, this library support importing CSV data. 

We strongly encourage collaboration - please free to fork this repo and tweak the code base to work for you own data, and make pull requests if you have suggestions on how to enhance the overall experience, add new features, or improve general performance.

## Set Up

In [17]:
%pip install -q --upgrade "Labelbox[data]"
%pip install -q pandas


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import labelbox as lb
import labelbox.types as lb_types
import uuid
import csv
import pandas

## API Key and Client
Provide a valid API key below to connect to the Labelbox client properly. For more information, please review the [Create API Key](https://docs.labelbox.com/reference/create-api-key) guide.

In [19]:
API_KEY = None
client = lb.Client(api_key=API_KEY)

## Create/Select Example Project

The below steps will setup a project that can be used for this demo. Please feel free to comment out the below code block and uncomment the code block to receive your own project directly. For more information on this set up visit our quick start guide.

### Create Project

In [20]:
# Create dataset with image data row
global_key = str(uuid.uuid4())

test_img_url = {
    "row_data":
        "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key":
        global_key,
}

dataset = client.create_dataset(name="image-demo-dataset")
task = dataset.create_data_rows([test_img_url])
task.wait_till_done()
print("Errors:", task.errors)
print("Failed data rows:", task.failed_data_rows)

# Create ontology
ontology_builder = lb.OntologyBuilder(
    classifications=[  # List of Classification objects
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="radio_question",
            options=[
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer"),
            ],
        ),
        lb.Classification(
            class_type=lb.Classification.Type.CHECKLIST,
            name="checklist_question",
            options=[
                lb.Option(value="first_checklist_answer"),
                lb.Option(value="second_checklist_answer"),
            ],
        ),
        lb.Classification(class_type=lb.Classification.Type.TEXT,
                          name="free_text"),
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="nested_radio_question",
            options=[
                lb.Option(
                    "first_radio_answer",
                    options=[
                        lb.Classification(
                            class_type=lb.Classification.Type.RADIO,
                            name="sub_radio_question",
                            options=[lb.Option("first_sub_radio_answer")],
                        )
                    ],
                )
            ],
        ),
    ],
    tools=[  # List of Tool objects
        lb.Tool(tool=lb.Tool.Type.BBOX, name="bounding_box"),
        lb.Tool(
            tool=lb.Tool.Type.BBOX,
            name="bbox_with_radio_subclass",
            classifications=[
                lb.Classification(
                    class_type=lb.Classification.Type.RADIO,
                    name="sub_radio_question",
                    options=[lb.Option(value="first_sub_radio_answer")],
                ),
            ],
        ),
    ],
)

ontology = client.create_ontology(
    "Image CSV Demo Ontology",
    ontology_builder.asdict(),
    media_type=lb.MediaType.Image,
)

# Set up project and connect ontology
project = client.create_project(name="Image Annotation Import Demo",
                                media_type=lb.MediaType.Image)
project.setup_editor(ontology)

# Send data row towards our project
batch = project.create_batch(
    "image-demo-batch",
    global_keys=[
        global_key
    ],  # paginated collection of data row objects, list of data row ids or global keys
    priority=1,
)

print(f"Batch: {batch}")

# Create a label and imported it towards our project
radio_annotation = lb_types.ClassificationAnnotation(
    name="radio_question",
    value=lb_types.Radio(answer=lb_types.ClassificationAnswer(
        name="second_radio_answer")),
)
checklist_annotation = lb_types.ClassificationAnnotation(
    name="checklist_question",
    value=lb_types.Checklist(answer=[
        lb_types.ClassificationAnswer(name="first_checklist_answer"),
        lb_types.ClassificationAnswer(name="second_checklist_answer"),
    ]),
)
text_annotation = lb_types.ClassificationAnnotation(
    name="free_text",
    value=lb_types.Text(answer="sample text"),
)
nested_radio_annotation = lb_types.ClassificationAnnotation(
    name="nested_radio_question",
    value=lb_types.Radio(answer=lb_types.ClassificationAnswer(
        name="first_radio_answer",
        classifications=[
            lb_types.ClassificationAnnotation(
                name="sub_radio_question",
                value=lb_types.Radio(answer=lb_types.ClassificationAnswer(
                    name="first_sub_radio_answer")),
            )
        ],
    )),
)
bbox_annotation = lb_types.ObjectAnnotation(
    name="bounding_box",
    value=lb_types.Rectangle(
        start=lb_types.Point(x=1690, y=977),
        end=lb_types.Point(x=1915, y=1307),
    ),
)

label = []
annotations = [
    radio_annotation,
    nested_radio_annotation,
    checklist_annotation,
    text_annotation,
    bbox_annotation,
]

label.append(
    lb_types.Label(data={"global_key": global_key}, annotations=annotations))

upload_job = lb.LabelImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="label_import_job" + str(uuid.uuid4()),
    labels=label,
)

upload_job.wait_until_done()
print("Errors:", upload_job.errors)
print("Status of uploads: ", upload_job.statuses)

Errors: None
Failed data rows: None
Batch: <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2024-06-03 12:11:18+00:00",
    "name": "image-demo-batch",
    "size": 1,
    "uid": "621533f0-21a2-11ef-a157-aba0c2cf595f",
    "updated_at": "2024-06-03 12:11:18+00:00"
}>
Errors: []
Status of uploads:  [{'uuid': '27ec3cab-543b-43fa-b9e2-edca4b57c3ed', 'dataRow': {'id': 'clwyxjdbw2hor0887c048u5tx', 'globalKey': 'bc378cff-33fa-4732-8cf8-089d7f0c9361'}, 'status': 'SUCCESS'}, {'uuid': '45306e29-cd9c-43de-8aaf-8a639f3e2a5d', 'dataRow': {'id': 'clwyxjdbw2hor0887c048u5tx', 'globalKey': 'bc378cff-33fa-4732-8cf8-089d7f0c9361'}, 'status': 'SUCCESS'}, {'uuid': 'ebb06054-274b-465d-a684-07ee32bf4b50', 'dataRow': {'id': 'clwyxjdbw2hor0887c048u5tx', 'globalKey': 'bc378cff-33fa-4732-8cf8-089d7f0c9361'}, 'status': 'SUCCESS'}, {'uuid': 'e9d0197b-7852-431e-8d60-46d379708091', 'dataRow': {'id': 'clwyxjdbw2hor0887c048u5tx', 'globalKey': 'bc378cff-33fa-

### Select Project

In [21]:
# PROJECT_ID = None
# project = client.get_project(PROJECT_ID)

## CSV Format Overview

In order to convert our Labelbox JSON data to a format more CSV friendly, we must first define the needed structure of our JSON. A common format that is versatile for both the built-in Python CSV writer and Pandas is as follows: 

```python
[
    {"<column_1>":"<answer_1", "<column_2>":"<answer_2" ..},
    
    {"<column_1>":"<answer_1", "<column_2>":"<answer_2" ..},
    ..
]
```

Essentially, we need to get our JSON data towards a list of Python dictionaries with each Python dictionary representing one row, each key representing a column and each value being an individual cell of our CSV table. Once we have our data to this format it is trivial to create Pandas DataFrames or write our CSV file. The tricky part is getting Labelbox export JSON towards this format.

## Labelbox JSON Format

Labelbox JSON format is centralized at the individual data row of your export. This format allows expandability when things evolve and provides a centralized view of fields such as metadata or data row details. The main labels are located inside the projects key and can be nested which can make it difficult to parse. For complete samples of our project export format visit our export quick reference page. 

To get Labelbox export JSON format to our CSV format we established, we must do the following:

1. Establish our base data row columns (project_id, data_row_id, global_key etc)
2. Create our columns for label fields (label detail and annotations we care about)
3. Define our functions and strategy used to parse through our data
4. Export our data
5. Navigate to the label level of each data row
6. Include any objects or tools in our format
7. Deal with nested classifications 

## Step 1: Establish our Base Columns

We first establish our base columns that typically represent individual data row details. Typically, this columns information can be received from within one or two levels of a Labelbox export per data row. 

Please feel free to modify the below columns if you want to include more. You will need to update the code later in this guide to pick up any additional columns.

In [22]:
data_row_base_columns = [
    "Data Row ID",
    "Global Key",
    "External ID",
    "Project ID",
]

## Step 2: Create our Columns for Label Fields

In this step, we first start by defining our label details base columns we want to include in our CSV. In this case, we will use the following:

In [23]:
label_base_columns = ["Label ID", "Created By", "Skipped"]

We then need to obtain the annotation we want to include in our columns. The order of our list matters since that is the order of what our columns will be presented. You can approach getting the annotations in a list in a number of ways including hard defining the columns. For the sack of simplicity our column names for our annotations needs to match the name of the actual feature of our export data. If you wish to have the column names defined differently you will have to do some type of mapping. In the code below, I will be recursively going through the ontology we created to get our column names based on the name of the features. In the next step of this guide we will provide more information on recursion in context of parsing through JSON or Python dictionaries.

In [24]:
def get_classification_features(classifications: list, class_list=[]) -> None:
    """Finds classification features inside an ontology recursively and returns them in a list"""
    for classification in classifications:
        if "name" in classification:
            class_list.append(classification["instructions"])
        if "options" in classification:
            get_classification_features(classification["options"], class_list)
    return class_list


def get_tool_features(tools: list) -> None:
    """Creates list of tool names from ontology"""
    tool_list = []
    for tool in tools:
        tool_list.append(tool["name"])
    return tool_list

In [25]:
# Get ontology from project and normalized towards python dictionary
ontology = project.ontology().normalized

class_annotation_columns = get_classification_features(
    ontology["classifications"])

tool_annotation_columns = get_tool_features(ontology["tools"])

## Step 3: Define our Functions and Strategy used to Parse through our Data

Now that we have our columns defined we need to come up with a strategy of navigating our export data. Review this [sample export](https://docs.labelbox.com/reference/export-image-annotations#sample-project-export) to follow along. While creating our columns it is always best to first check if a key exists in your data row before populating a column this is especially import for optional fields. In this demo, we will populate the value `None` for anything not present which will result in a blank cell our our CSV.


### Data Row Detail Base Columns
The data row details can be access within a depth of one or two keys. Below is a function we will use the access the columns we defined. The parameters are the data row itself, dictionary row that will be used to make our list and our base columns list.

In [ ]:
def get_base_data_row_columns(data_row: dict[str:str], csv_row: dict[str:str], base_columns: list[str]) -> dict[str:str]:
    for base_column in base_columns:
        if base_column == "Data Row ID":
            csv_row[base_column] = data_row["data_row"]["id"]
            
        elif base_column == "Global Key":
            if "global_key" in data_row["data_row"]: # Check if global key exists
                csv_row[base_column] = data_row["data_row"]["global_key"]
            else: 
                csv_row[base_column] = None # If global key does not exist on data row set cell to None. This will create a blank cell on your csv
                
        elif base_column == "External ID":
            if "external_id" in data_row["data_row"]: # Check if external_id exists
                csv_row[base_column] = data_row["data_row"]["external_id"]
            else:
                csv_row[base_column] = None # If external id does not exist on data row set cell to None. This will create a blank cell on your csv
                
        elif base_column == "Project ID":
            csv_row["projects"] = project.uid
    return csv_row

### Label Detail Base Columns
The label details are similar to data row details but they exist at a label level of our export. Later in the guide we will demonstrate how to get our exported data row at this level. The function below shows the process of obtaining the details we defined above. The parameters are the label, the dictionary row that we will be modifying and the label detail column list we created.

In [ ]:
def get_base_label_columns(label: dict[str:str], csv_row: dict[str:str], label_base_columns: list[str]) -> dict[str:str]:
    for label_base_column in label_base_columns:
        if label_base_column == "Label ID":
            csv_row["label_base_columns"] = label["id"]
            
        elif label_base_columns == "Created By":
            if "label_details" in label: # Check if label details is present. This field can be omitted in export
                csv_row["created_by"] = label_base_columns["label_details"]["created_by"]
            else:
                csv_row["created_by"] = None
                
        elif label_base_column == "Skipped":
            if "performance_details" in label: # Check if performance details are present. This field can be omitted in export. 
                csv_row["Skipped"] = label["performance_details"]["skipped"]
            else:
                csv_row["Skipped"] = None
                
    return csv_row

### Label Annotation Columns
The label annotations are the final columns we will need to obtain. The approach of obtaining these fields are more challenging then the approach we made for our detail columns. If we attempt to obtain the fields with conditional statements and hard defined paths we will quickly run into issues as each label can have annotations in different orders or annotations can be at different depths or not present at all. This will quickly create a mess especially when we want our methods to work for more then one ontology. The best and cleanest way of obtaining these annotations inside our export data is through recursive function.

#### Recursion
A recursive function can be defined as a routine that calls itself directly or indirectly. They solve a problems by solving smaller instances of the same problem. This technique is commonly used in programming to solve problems that can be broken down into simpler, similar subproblem. Our sub problem in this case is obtaining the each individual annotations. A recursive function is divided into two components:

- *Base case:* This is a termination condition that prevents the function from calling itself indefinitely.

- *Recursive case:* In the recursive case, the function calls itself with the modified arguments. The recursive case should move closer to the base case with each iteration.



In [27]:
from pprint import pprint

export_task = project.export()
export_task.wait_till_done()

stream = export_task.get_buffered_stream()

if export_task.has_result():

    for data_row in stream:
        pprint(data_row.json, indent=0.5)

{'data_row': {'global_key': 'bc378cff-33fa-4732-8cf8-089d7f0c9361',
            'id': 'clwyxjdbw2hor0887c048u5tx',
            'row_data': 'https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg'},
'media_attributes': {'asset_type': 'image',
                    'exif_rotation': '1',
                    'height': 1707,
                    'mime_type': 'image/jpeg',
                    'width': 2560},
'projects': {'clwyxjn8e00fy07y3du1zc098': {'labels': [{'annotations': {'classifications': [{'feature_id': 'clwyxjvs81o1p146z44vajhz3',
                                                                                     'feature_schema_id': 'clwyxjmcl04q6070ogcg3hbw8',
                                                                                     'name': 'radio_question',
                                                                                     'radio_answer': {'classifications': [],
                                       